In [ ]:
!pip install gspread
!pip install oauth2client

In [ ]:
import gspread
import boto3
import json
import numpy as np
import pandas as pd
from oauth2client.service_account import ServiceAccountCredentials
# define the scope
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
s3 = boto3.client('s3')
#s3.download_file('datascience-firstviewsheets', 'sheets_creds.json', 'sheets_creds.json')
cred_obj = s3.get_object(Bucket='datascience-hbomax-jialuyou', Key='us-forecasting-52ecbdfeada1.json') ## Bucket name and file Key name json - 1-time
cred_dict = json.loads(cred_obj['Body'].read().decode('utf-8'))
# add credentials to the account
creds = ServiceAccountCredentials.from_json_keyfile_dict(cred_dict)
# authorize the clientsheet 
client = gspread.authorize(creds)
sheet = client.open('content_first_view_weekly_projection') #share sheet with that account
sheet_instance = sheet.get_worksheet(0)
records_data = pd.DataFrame(sheet_instance.get_all_values())

In [ ]:
pd.DataFrame(records_data).head()

In [ ]:
import pandas as pd
import json
import csv
from google.oauth2 import service_account
import statsmodels.formula.api as sm  
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pandas.tseries.offsets import MonthEnd, Week
from datetime import timedelta
from datetime import datetime
import gspread
import boto3
from oauth2client.service_account import ServiceAccountCredentials
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from abc import ABCMeta, abstractmethod
import snowflake.connector


In [ ]:
from google.oauth2 import service_account

In [ ]:
DB_NAME = "MAX_DEV"
SCHEMA = "WORKSPACE"
## Credentials
SF_CREDS = 'datascience-max-dev-sagemaker-notebooks'
class Credentials(metaclass=ABCMeta):
    pass
    
    
class SSMPSCredentials(Credentials):
    def __init__(self, secretid: str):
        self._secretid = secretid
        self._secrets = {}
        
    def get_keys(self):
        """
        credential fetching 
        """
        _aws_sm_args = {'service_name': 'secretsmanager', 'region_name': 'us-east-1'}
        secrets_client = boto3.client(**_aws_sm_args)
        get_secret_value_response = secrets_client.get_secret_value(SecretId=self._secretid)
        return get_secret_value_response
    
    
class BaseConnector(metaclass=ABCMeta):
    @abstractmethod
    def connect(self):
        raise NotImplementedError
        
class SnowflakeConnector(BaseConnector):
    def __init__(self, credentials: Credentials):
        keys = credentials.get_keys()
        self._secrets = json.loads(keys.get('SecretString', "{}"))
    def connect(self, dbname: str, schema: str = 'DEFAULT'):
        ctx = snowflake.connector.connect(
            user=self._secrets['login_name'],
            password=self._secrets['login_password'],
            account=self._secrets['account'],
            warehouse=self._secrets['warehouse'],
            database=dbname,
            schema=schema
        )
        return ctx
connector = SnowflakeConnector(SSMPSCredentials(SF_CREDS))
con = connector.connect(dbname=DB_NAME, schema=SCHEMA)


In [ ]:
query = """SELECT * FROM TABLE XXX LIMIT 1000"""

In [ ]:
df = pd.read_sql(query, con)